# 🦷 NHANES Periodontitis: External Validation & Final Analyses

**Notebook 01: External Validation, CIs, DCA, Prevalence Reconciliation**

This notebook completes the analysis pipeline for medRxiv submission:

1. **Section 23:** External Validation (NHANES 2009-2010)
2. **Section 24:** Prevalence Reconciliation
3. **Section 25:** Bootstrap 95% Confidence Intervals
4. **Section 26:** Permutation Tests (Run & Save)
5. **Section 27:** Decision Curve Analysis (DCA)
6. **Section 28:** KNHANES Scaffold (Optional)

---


## Section 23: Environment Setup & Imports


In [5]:
"""
Section 23.0: Environment Setup
===============================
"""
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    roc_curve, precision_recall_curve, confusion_matrix
)
from sklearn.calibration import calibration_curve
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import json
from datetime import datetime
import os
import sys

# Find project root
def find_project_root(marker="configs/config.yaml"):
    here = Path.cwd()
    for candidate in [here] + list(here.parents):
        if (candidate / marker).exists():
            return candidate
    raise FileNotFoundError(f"Could not locate {marker}")

BASE_DIR = find_project_root()
os.chdir(BASE_DIR)
print(f"Working directory: {Path.cwd()}")

# Add src to path
sys.path.insert(0, str(BASE_DIR / 'src'))

# Import custom modules
from ps_plot import set_style, get_palette, save_figure, PERIOSPOT_BLUE, PERIOSPOT_RED
from labels import label_periodontitis
from stats_utils import permutation_test_auc, pairwise_permutation_tests

# Set style
set_style()
palette = get_palette()

# Define paths
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
FIGURES_DIR = BASE_DIR / 'figures'
RESULTS_DIR = BASE_DIR / 'results'

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Section 23.0: Environment setup complete")


Working directory: /Users/franciscoteixeirabarbosa/Dropbox/Random_scripts/nhanes_periodontitis_ml
✅ Section 23.0: Environment setup complete


## Section 23.1: Load Training Data and Model Parameters


In [6]:
"""
Section 23.1: Load Training Data & Tuned Parameters
===================================================
"""
# Load training features (cleaned, with missing indicators)
df_train_full = pd.read_parquet(PROCESSED_DIR / 'features_cleaned.parquet')
print(f"Training data: {df_train_full.shape}")

# Load tuned hyperparameters from v1.3
with open(RESULTS_DIR / 'xgboost_results.json', 'r') as f:
    xgb_results = json.load(f)
with open(RESULTS_DIR / 'catboost_results.json', 'r') as f:
    cat_results = json.load(f)
with open(RESULTS_DIR / 'lightgbm_results.json', 'r') as f:
    lgbm_results = json.load(f)

# Extract best params
tuned_xgb_params = xgb_results['best_params']
tuned_cat_params = cat_results['best_params']
tuned_lgbm_params = lgbm_results['best_params']

print("\n✅ Loaded tuned hyperparameters")

# Define feature lists (PRIMARY MODEL - 29 features, no reverse-causality)
CONTINUOUS_FEATURES = ['age', 'bmi', 'waist_cm', 'waist_height', 'height_cm',
                       'systolic_bp', 'diastolic_bp', 'glucose', 'triglycerides', 'hdl']
BINARY_FEATURES = ['sex', 'education', 'smoking', 'alcohol',
                   'smoke_current', 'smoke_former', 'alcohol_current']
MISSING_INDICATORS = ['bmi_missing', 'systolic_bp_missing', 'diastolic_bp_missing',
                      'glucose_missing', 'triglycerides_missing', 'hdl_missing',
                      'smoking_missing', 'alcohol_missing',
                      'waist_cm_missing', 'waist_height_missing', 'height_cm_missing',
                      'alcohol_current_missing']

# Reverse-causality features (EXCLUDED from primary model)
REVERSE_CAUSALITY = ['dental_visit', 'floss_days', 'mobile_teeth', 'floss_days_missing']

# Primary model features
ALL_FEATURES_PRIMARY = CONTINUOUS_FEATURES + BINARY_FEATURES + MISSING_INDICATORS
ALL_FEATURES_PRIMARY = [f for f in ALL_FEATURES_PRIMARY if f not in REVERSE_CAUSALITY]
ALL_FEATURES_PRIMARY = [f for f in ALL_FEATURES_PRIMARY if f in df_train_full.columns]
print(f"\nPrimary model features: {len(ALL_FEATURES_PRIMARY)}")

# Monotonic constraints
MONO_INCREASING = ['age', 'bmi', 'waist_cm', 'waist_height',
                   'systolic_bp', 'diastolic_bp', 'glucose', 'triglycerides']
MONO_DECREASING = ['hdl']

print("✅ Section 23.1: Training data and parameters loaded")


Training data: (9379, 37)

✅ Loaded tuned hyperparameters

Primary model features: 29
✅ Section 23.1: Training data and parameters loaded


## Section 23.2: Download & Process NHANES 2009-2010 Data


In [ ]:
"""
Section 23.2: Download NHANES 2009-2010 Data
============================================
Same data sources and variable mappings as 2011-2014
"""
import requests

# NHANES 2009-2010 URLs (UPDATED: New CDC URL structure as of 2024)
# Old format: /Nchs/Nhanes/2009-2010/DEMO_F.XPT
# New format: /Nchs/Data/Nhanes/Public/2009/DataFiles/DEMO_F.xpt
BASE_URL_2009 = 'https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2009/DataFiles'

NHANES_2009_2010 = {
    'demographics': f'{BASE_URL_2009}/DEMO_F.xpt',
    'body_measures': f'{BASE_URL_2009}/BMX_F.xpt',
    'blood_pressure': f'{BASE_URL_2009}/BPX_F.xpt',
    'smoking': f'{BASE_URL_2009}/SMQ_F.xpt',
    'alcohol': f'{BASE_URL_2009}/ALQ_F.xpt',
    'oral_health_questionnaire': f'{BASE_URL_2009}/OHQ_F.xpt',
    'periodontal_exam': f'{BASE_URL_2009}/OHXPER_F.xpt',
    'glucose': f'{BASE_URL_2009}/GLU_F.xpt',
    'triglycerides': f'{BASE_URL_2009}/TRIGLY_F.xpt',
    'hdl_cholesterol': f'{BASE_URL_2009}/HDL_F.xpt',
}

# Create directory
cycle_dir = RAW_DIR / '2009_2010'
cycle_dir.mkdir(parents=True, exist_ok=True)

def download_and_convert_xpt(url, output_path, force_redownload=False):
    """Download XPT file and convert to parquet"""
    # Check if parquet already exists and is valid
    if output_path.exists() and not force_redownload:
        try:
            df = pd.read_parquet(output_path)
            if len(df) > 0:
                print(f"  ✓ Already exists: {output_path.name} ({len(df)} rows)")
                return df
        except:
            pass  # File is corrupt, re-download
    
    # Check if XPT exists and convert
    xpt_path = output_path.with_suffix('.xpt')
    if xpt_path.exists():
        try:
            df = pd.read_sas(xpt_path)
            if len(df) > 0:
                df.to_parquet(output_path)
                print(f"  ✓ Converted from XPT: {output_path.name} ({len(df)} rows)")
                return df
        except Exception as e:
            print(f"  ⚠️ XPT file corrupt, re-downloading: {e}")
    
    # Download fresh
    print(f"  ↓ Downloading: {url.split('/')[-1]}")
    response = requests.get(url, timeout=60)
    response.raise_for_status()
    
    # Check file size
    if len(response.content) < 50000:  # Less than 50KB is suspicious
        print(f"  ⚠️ Warning: File seems small ({len(response.content)} bytes)")
    
    # Save as XPT first
    with open(xpt_path, 'wb') as f:
        f.write(response.content)
    
    # Convert to parquet
    df = pd.read_sas(xpt_path)
    df.to_parquet(output_path)
    
    print(f"  ✓ Downloaded and converted: {output_path.name} ({len(df)} rows)")
    return df

print("📥 Downloading/Converting NHANES 2009-2010 data...\n")

dfs_0910 = {}
for name, url in NHANES_2009_2010.items():
    output_path = cycle_dir / f"{name}.parquet"
    try:
        dfs_0910[name] = download_and_convert_xpt(url, output_path)
    except Exception as e:
        print(f"  ✗ Error with {name}: {e}")

print("\n✅ Section 23.2: 2009-2010 data downloaded/converted")
print(f"\n📊 Data summary:")
for name, df in dfs_0910.items():
    print(f"   {name}: {len(df)} rows")


📥 Downloading/Converting NHANES 2009-2010 data...

  ↓ Downloading: DEMO_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with demographics: Header record is not an XPORT file.
  ↓ Downloading: BMX_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with body_measures: Header record is not an XPORT file.
  ↓ Downloading: BPX_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with blood_pressure: Header record is not an XPORT file.
  ↓ Downloading: SMQ_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with smoking: Header record is not an XPORT file.
  ↓ Downloading: ALQ_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with alcohol: Header record is not an XPORT file.
  ↓ Downloading: OHQ_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with oral_health_questionnaire: Header record is not an XPORT file.
  ↓ Downloading: OHXPER_F.XPT
  ⚠️ Warning: File seems small (20905 bytes)
  ✗ Error with periodontal_exam: Header recor

## Section 23.3: Merge and Label 2009-2010 Data


In [8]:
"""
Section 23.3: Merge 2009-2010 Components & Apply CDC/AAP Labels
===============================================================
"""
# Reload from parquet
demo = pd.read_parquet(cycle_dir / 'demographics.parquet')
body = pd.read_parquet(cycle_dir / 'body_measures.parquet')
bp = pd.read_parquet(cycle_dir / 'blood_pressure.parquet')
smq = pd.read_parquet(cycle_dir / 'smoking.parquet')
alq = pd.read_parquet(cycle_dir / 'alcohol.parquet')
ohq = pd.read_parquet(cycle_dir / 'oral_health_questionnaire.parquet')
perio = pd.read_parquet(cycle_dir / 'periodontal_exam.parquet')
glu = pd.read_parquet(cycle_dir / 'glucose.parquet')
trig = pd.read_parquet(cycle_dir / 'triglycerides.parquet')
hdl_df = pd.read_parquet(cycle_dir / 'hdl_cholesterol.parquet')

print(f"Demographics: {len(demo)}")
print(f"Periodontal: {len(perio)}")

# Merge all on SEQN
df_0910 = demo.merge(body, on='SEQN', how='left')
df_0910 = df_0910.merge(bp, on='SEQN', how='left')
df_0910 = df_0910.merge(smq, on='SEQN', how='left')
df_0910 = df_0910.merge(alq, on='SEQN', how='left')
df_0910 = df_0910.merge(ohq, on='SEQN', how='left')
df_0910 = df_0910.merge(perio, on='SEQN', how='left')
df_0910 = df_0910.merge(glu, on='SEQN', how='left')
df_0910 = df_0910.merge(trig, on='SEQN', how='left')
df_0910 = df_0910.merge(hdl_df, on='SEQN', how='left')

print(f"\nMerged: {len(df_0910)} rows")

# Filter to adults 30+
df_0910 = df_0910[df_0910['RIDAGEYR'] >= 30].copy()
print(f"Adults 30+: {len(df_0910)} rows")

# Apply CDC/AAP periodontitis labels
print("\n🦷 Applying CDC/AAP classification...")
df_0910_labeled = label_periodontitis(df_0910)

if df_0910_labeled is not None:
    # Save labeled data
    df_0910_labeled.to_parquet(PROCESSED_DIR / '2009_2010_labeled.parquet')
    print(f"\n✅ Labeled: {len(df_0910_labeled)} participants")
    print(f"   Periodontitis prevalence: {df_0910_labeled['has_periodontitis'].mean()*100:.1f}%")
else:
    print("⚠️ Labeling failed - check periodontal exam columns")

print("\n✅ Section 23.3: 2009-2010 data merged and labeled")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/franciscoteixeirabarbosa/Dropbox/Random_scripts/nhanes_periodontitis_ml/data/raw/2009_2010/demographics.parquet'

## Section 23.4: Build Features for 2009-2010 (Same Pipeline as Training)


In [ ]:
"""
Section 23.4: Build Features for 2009-2010
==========================================
Apply EXACT same feature engineering as training data
"""
# Load labeled data
df_0910 = pd.read_parquet(PROCESSED_DIR / '2009_2010_labeled.parquet')

# Build features (same logic as Section 6 in notebook 00)
df_ext = pd.DataFrame(index=df_0910.index)

# Demographics
df_ext['age'] = df_0910['RIDAGEYR']
df_ext['sex'] = (df_0910['RIAGENDR'] == 1).astype(int)  # 1=Male
df_ext['education'] = (df_0910['DMDEDUC2'] >= 4).astype(int)  # >=High school

# Metabolic
df_ext['bmi'] = df_0910['BMXBMI']
df_ext['waist_cm'] = df_0910['BMXWAIST']
df_ext['height_cm'] = df_0910['BMXHT']
df_ext['waist_height'] = df_0910['BMXWAIST'] / df_0910['BMXHT']
df_ext['systolic_bp'] = df_0910['BPXSY1']
df_ext['diastolic_bp'] = df_0910['BPXDI1']
df_ext['glucose'] = df_0910['LBXGLU'] if 'LBXGLU' in df_0910.columns else np.nan
df_ext['triglycerides'] = df_0910['LBXTR'] if 'LBXTR' in df_0910.columns else np.nan
df_ext['hdl'] = df_0910['LBDHDD'] if 'LBDHDD' in df_0910.columns else np.nan

# Behaviors - Smoking (3-level)
df_ext['smoking'] = df_0910['SMQ040'].apply(
    lambda x: 1 if x in [1, 2] else (0 if x == 3 else np.nan)
)
df_ext['smoke_current'] = (
    (df_0910['SMQ020'] == 1) & (df_0910['SMQ040'].isin([1, 2]))
).astype(int)
df_ext['smoke_former'] = (
    (df_0910['SMQ020'] == 1) & (df_0910['SMQ040'] == 3)
).astype(int)

# Alcohol
df_ext['alcohol'] = df_0910['ALQ101'].apply(
    lambda x: 1 if x == 1 else (0 if x == 2 else np.nan)
)
df_ext['alcohol_current'] = df_0910['ALQ110'].apply(
    lambda x: 1 if x == 1 else (0 if x == 2 else np.nan)
) if 'ALQ110' in df_0910.columns else np.nan

# Oral health (for secondary model - excluded from primary)
df_ext['dental_visit'] = (df_0910['OHQ030'] <= 2).astype(int) if 'OHQ030' in df_0910.columns else np.nan
df_ext['mobile_teeth'] = (df_0910['OHQ680'] == 1).astype(int) if 'OHQ680' in df_0910.columns else np.nan
df_ext['floss_days'] = df_0910['OHQ620'] if 'OHQ620' in df_0910.columns else np.nan

# Target
df_ext['has_periodontitis'] = df_0910['has_periodontitis']
df_ext['severity'] = df_0910['severity']

# Add missing indicators
for feat in ['bmi', 'systolic_bp', 'diastolic_bp', 'glucose', 'triglycerides', 'hdl',
             'smoking', 'alcohol', 'waist_cm', 'waist_height', 'height_cm', 'alcohol_current',
             'floss_days']:
    if feat in df_ext.columns:
        df_ext[f'{feat}_missing'] = df_ext[feat].isna().astype(int)

# Data cleaning (same as training)
if 'diastolic_bp' in df_ext.columns:
    df_ext['diastolic_bp'] = df_ext['diastolic_bp'].clip(40, 120)

if 'triglycerides' in df_ext.columns:
    p99 = df_ext['triglycerides'].quantile(0.99)
    df_ext['triglycerides'] = df_ext['triglycerides'].clip(upper=p99)

# Drop rows without target
df_ext = df_ext.dropna(subset=['has_periodontitis'])

print(f"External test set: {len(df_ext)} participants")
print(f"Periodontitis prevalence: {df_ext['has_periodontitis'].mean()*100:.1f}%")
print(f"\nFeatures available: {df_ext.shape[1]}")

# Save
df_ext.to_parquet(PROCESSED_DIR / '2009_2010_features.parquet')

print("\n✅ Section 23.4: External features built")


## Section 23.5-23.7: External Validation (Train on 2011-2014, Evaluate on 2009-2010)

The following cells will:
1. Train the primary ensemble on full 2011-2014 data
2. Generate calibrated predictions on 2009-2010
3. Compute metrics with 95% CIs
4. Generate ROC/PR/Calibration plots


In [ ]:
# TODO: Run this cell after downloading and processing 2009-2010 data
# See notebook 00 for full implementation details

print("="*70)
print("📊 EXTERNAL VALIDATION WORKFLOW")
print("="*70)
print("""
Steps to complete:
1. Run cells 23.2-23.4 to download and process 2009-2010 data
2. Train ensemble on 2011-2014 (using tuned params from notebook 00)
3. Evaluate on 2009-2010
4. Compute metrics with bootstrap 95% CIs
5. Save results to results/external_0910_metrics.json
6. Generate figures/external_roc_pr_calibration.png

Key outputs expected:
- AUC-ROC with 95% CI
- PR-AUC with 95% CI
- Brier score with 95% CI
- Operating point metrics at t=0.35 (rule-out) and t=0.65 (balanced)
""")
print("="*70)


---
## Section 24: Prevalence Reconciliation


In [ ]:
"""
Section 24: Prevalence Reconciliation
=====================================
Compare our prevalence to CDC/AAP published estimates
"""
# Load all labeled datasets
cycles = {
    '2011-2012': PROCESSED_DIR / '2011_2012_labeled.parquet',
    '2013-2014': PROCESSED_DIR / '2013_2014_labeled.parquet',
}

# Check if 2009-2010 exists
if (PROCESSED_DIR / '2009_2010_labeled.parquet').exists():
    cycles['2009-2010'] = PROCESSED_DIR / '2009_2010_labeled.parquet'

prevalence_data = []

print("📊 PREVALENCE RECONCILIATION")
print("=" * 60)

for cycle, path in cycles.items():
    if path.exists():
        df = pd.read_parquet(path)
        df = df[df['has_periodontitis'].notna()]
        
        total = len(df)
        perio = df['has_periodontitis'].sum()
        prev = perio / total * 100
        
        severe = (df['severity'] == 'severe').sum() / total * 100 if 'severity' in df.columns else np.nan
        moderate = (df['severity'] == 'moderate').sum() / total * 100 if 'severity' in df.columns else np.nan
        mild = (df['severity'] == 'mild').sum() / total * 100 if 'severity' in df.columns else np.nan
        
        prevalence_data.append({
            'cycle': cycle, 'n': total, 'prevalence': prev,
            'severe': severe, 'moderate': moderate, 'mild': mild
        })
        
        print(f"\n{cycle}:")
        print(f"  N = {total:,}")
        print(f"  Total periodontitis: {prev:.1f}%")
        if not np.isnan(severe):
            print(f"    - Severe: {severe:.1f}%, Moderate: {moderate:.1f}%, Mild: {mild:.1f}%")

# CDC Reference
print("\n" + "=" * 60)
print("\n📚 CDC Reference (Eke et al. 2015, NHANES 2009-2012):")
print("  Adults 30+ with periodontitis: 47.2%")
print("    - Severe: 8.9%, Moderate: 30.0%, Mild: 8.7%")

print("\n⚠️ RECONCILIATION NOTE:")
print("  Our higher prevalence (~68%) vs CDC (~47%) reflects:")
print("  1. Only participants with FULL periodontal exam")
print("  2. CDC includes partial/edentulous participants")

# Save results
prevalence_results = {
    'our_estimates': prevalence_data,
    'cdc_reference': {
        'source': 'Eke et al. 2015, J Periodontol',
        'total_periodontitis': 47.2, 'severe': 8.9, 'moderate': 30.0, 'mild': 8.7
    },
    'reconciliation_note': "Higher prevalence reflects full periodontal exam inclusion criteria.",
    'timestamp': datetime.now().isoformat()
}

with open(RESULTS_DIR / 'prevalence_check.json', 'w') as f:
    json.dump(prevalence_results, f, indent=2)

print(f"\n✅ Saved: results/prevalence_check.json")


---
## Section 25-27: Additional Analyses (CIs, Permutation Tests, DCA)

**Note:** The following analyses require running the full training pipeline from notebook 00. They should be run after all models are trained and OOF predictions are available.

### Section 25: Bootstrap 95% CIs
Compute confidence intervals for AUC, PR-AUC, Brier using 2000 stratified bootstrap resamples.

### Section 26: Permutation Tests  
Run paired permutation tests (10,000 iterations) on OOF predictions.

### Section 27: Decision Curve Analysis
Compute net benefit curves for the primary model.


In [ ]:
"""
Helper Functions for Sections 25-27
===================================
"""
def bootstrap_ci(y_true, y_score, metric_fn, n_bootstrap=2000, ci=0.95, seed=42):
    """Compute bootstrap confidence interval for a metric"""
    np.random.seed(seed)
    n = len(y_true)
    scores = []
    
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, n, replace=True)
        try:
            score = metric_fn(y_true[idx], y_score[idx])
            scores.append(score)
        except:
            continue
    
    scores = np.array(scores)
    alpha = (1 - ci) / 2
    return np.mean(scores), np.percentile(scores, alpha * 100), np.percentile(scores, (1 - alpha) * 100)

def decision_curve_analysis(y_true, y_prob, thresholds=None):
    """Compute net benefit for DCA"""
    if thresholds is None:
        thresholds = np.arange(0.01, 0.99, 0.01)
    
    n = len(y_true)
    prevalence = np.mean(y_true)
    results = []
    
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        tp = np.sum((y_pred == 1) & (y_true == 1))
        fp = np.sum((y_pred == 1) & (y_true == 0))
        
        net_benefit_model = (tp / n) - (fp / n) * (t / (1 - t))
        net_benefit_all = prevalence - (1 - prevalence) * (t / (1 - t))
        
        results.append({
            'threshold': t,
            'model': max(net_benefit_model, 0),
            'treat_all': max(net_benefit_all, 0),
            'treat_none': 0
        })
    
    return pd.DataFrame(results)

print("✅ Helper functions defined")


---
## Final Summary

After running all sections, you should have:

**Results files:**
- `results/external_0910_metrics.json` - External validation metrics
- `results/prevalence_check.json` - Prevalence reconciliation  
- `results/v13_primary_ci.json` - Bootstrap CIs
- `results/model_comp_permutation.json` - Permutation test results
- `results/decision_curve_primary.json` - DCA data

**Figures:**
- `figures/external_roc_pr_calibration.png` - External validation plots
- `figures/prevalence_barplot.png` - Prevalence comparison
- `figures/decision_curve_primary.png` - DCA curves

**Next steps for medRxiv:**
1. Update ARTICLE_DRAFT.md with external validation results
2. Add 95% CIs to all metrics tables
3. Insert DCA paragraph in Discussion
4. Add prevalence reconciliation note in Methods
5. Archive repo to Zenodo for DOI
6. Final proofread and submit!
